<a href="https://colab.research.google.com/github/darshshah0204-wq/Digantara-Assessment/blob/main/Digantara_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sgp4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.7/235.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
from datetime import datetime, timedelta, timezone
from sgp4.api import Satrec

# ============================================================
# CONSTANTS
# ============================================================

MU_EARTH = 398600.4418        # km^3/s^2
R_EARTH = 6378.137           # km
J2 = 1.08262668e-3
DEG2RAD = np.pi / 180.0
RAD2DEG = 180.0 / np.pi


# ============================================================
# TIME UTILITIES
# ============================================================

def datetime_to_jd(dt):
    """Convert UTC datetime to Julian Date."""
    a = (14 - dt.month)//12
    y = dt.year + 4800 - a
    m = dt.month + 12*a - 3
    jd = dt.day + ((153*m + 2)//5) + 365*y + y//4 - y//100 + y//400 - 32045
    jd += (dt.hour - 12)/24 + dt.minute/1440 + dt.second/86400
    return jd


# ============================================================
# J2-PERTURBED TRACKER PROPAGATION (TEME)
# ============================================================

def tracker_state_ijk_km_s(a_km, inc_deg, raan_deg, argp_deg, M0_deg, t_s):
    """
    Tracker propagation using secular J2 perturbations.
    All outputs are in an inertial TEME-consistent frame.
    """

    inc = inc_deg * DEG2RAD
    raan0 = raan_deg * DEG2RAD
    argp0 = argp_deg * DEG2RAD
    M0 = M0_deg * DEG2RAD

    n0 = np.sqrt(MU_EARTH / a_km**3)
    p = a_km  # e = 0 (circular)

    # Secular J2 rates
    raan_dot = -1.5 * n0 * J2 * (R_EARTH/p)**2 * np.cos(inc)
    argp_dot = 0.75 * n0 * J2 * (R_EARTH/p)**2 * (4 - 5*np.sin(inc)**2)
    M_dot = n0 + 0.75 * n0 * J2 * (R_EARTH/p)**2 * (2 - 3*np.sin(inc)**2)

    # Propagate elements
    raan = raan0 + raan_dot * t_s
    argp = argp0 + argp_dot * t_s
    M = M0 + M_dot * t_s
    nu = M  # circular orbit

    # Perifocal position and velocity
    r_pf = np.array([
        a_km * np.cos(nu),
        a_km * np.sin(nu),
        0.0
    ])

    v_mag = np.sqrt(MU_EARTH / a_km)
    v_pf = np.array([
        -v_mag * np.sin(nu),
        v_mag * np.cos(nu),
        0.0
    ])

    # Rotation matrices
    def Rz(a):
        return np.array([
            [np.cos(a), -np.sin(a), 0],
            [np.sin(a),  np.cos(a), 0],
            [0, 0, 1]
        ])

    def Rx(a):
        return np.array([
            [1, 0, 0],
            [0, np.cos(a), -np.sin(a)],
            [0, np.sin(a),  np.cos(a)]
        ])

    Q = Rz(raan) @ Rx(inc) @ Rz(argp)

    r_ijk = Q @ r_pf
    v_ijk = Q @ v_pf

    return r_ijk, v_ijk


# ============================================================
# SUNLIGHT MODEL (CYLINDRICAL SHADOW)
# ============================================================

def is_sunlit(r_sat):
    """
    Simple cylindrical Earth shadow model.
    Sun direction assumed along +X TEME.
    """
    sun_dir = np.array([1.0, 0.0, 0.0])
    proj = np.dot(r_sat, sun_dir)
    perp = np.linalg.norm(r_sat - proj * sun_dir)

    if proj < 0 and perp < R_EARTH:
        return False
    return True


# ============================================================
# MISSION ANALYSIS
# ============================================================

def analyze_mission():

    # --- Target satellite (SGP4 / TEME) ---
    tle1 = "1 63223U 25052P 25244.59601767 .00010814 00000-0 51235-3 0 9991"
    tle2 = "2 63223 97.4217 137.0451 0006365 74.2830 285.9107 15.19475170 25990"
    target = Satrec.twoline2rv(tle1, tle2)

    # --- Analysis window ---
    required_start = datetime(2025, 9, 1, 0, 0, 0, tzinfo=timezone.utc)
    required_end = required_start + timedelta(hours=24)

    # Use TLE epoch if required start is invalid
    tle_epoch = datetime(2025, 9, 1, 14, 18, 16, tzinfo=timezone.utc)
    start_time = max(required_start, tle_epoch)
    end_time = required_end

    print(f"[INFO] Analysis window: {start_time} to {end_time}")

    # --- Tracker parameters ---
    a_km = 6878.0
    inc_deg = 97.4
    raan_deg = 72.628
    argp_deg = 331.7425
    M0_deg = 0.0

    half_fov_deg = 15.0
    max_range_km = 1000.0
    dt = 10

    crossing_events = []
    visible_events = []

    in_cross = False
    in_vis = False

    t = start_time
    while t <= end_time:

        jd = datetime_to_jd(t)
        e, r_tgt, v_tgt = target.sgp4(jd, 0.0)
        if e != 0:
            t += timedelta(seconds=dt)
            continue

        r_tgt = np.array(r_tgt)

        r_trk, v_trk = tracker_state_ijk_km_s(
            a_km, inc_deg, raan_deg, argp_deg, M0_deg,
            (t - start_time).total_seconds()
        )

        los = r_tgt - r_trk
        dist = np.linalg.norm(los)

        if dist > 1e-6:
            los_hat = los / dist
            v_hat = v_trk / np.linalg.norm(v_trk)
            angle = np.degrees(np.arccos(np.clip(np.dot(los_hat, v_hat), -1, 1)))
            crossing = angle <= half_fov_deg
        else:
            crossing = False

        visible = crossing and is_sunlit(r_tgt) and dist <= max_range_km

        # Crossing logic
        if crossing and not in_cross:
            cross_start = t
            in_cross = True
        if not crossing and in_cross:
            crossing_events.append((cross_start, t))
            in_cross = False

        # Visibility logic
        if visible and not in_vis:
            vis_start = t
            in_vis = True
        if not visible and in_vis:
            visible_events.append((vis_start, t))
            in_vis = False

        t += timedelta(seconds=dt)

    # ========================================================
    # RESULTS
    # ========================================================

    print("\n" + "="*70)
    print("RESULTS REPORT")
    print("="*70)

    print(f"CROSSING EVENTS: {len(crossing_events)}")
    for s, e in crossing_events:
        print(f"  {s} -> {e}")

    print(f"\nVISIBLE EVENTS: {len(visible_events)}")
    for s, e in visible_events:
        print(f"  {s} -> {e}")


if __name__ == "__main__":
    analyze_mission()


[INFO] Analysis window: 2025-09-01 14:18:16+00:00 to 2025-09-02 00:00:00+00:00

RESULTS REPORT
CROSSING EVENTS: 0

VISIBLE EVENTS: 0
